# Generating heatmaps from paths

In [1]:
#%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 20, 16
#import matplotlib.pyplot as plt
#plt.style.use('ggplot')
#import matplotlib.cm as cm

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 2.7.12 64bit [GCC 5.4.0 20160609]
IPython 5.4.1
OS Linux 4.10.0 32 generic x86_64 with Ubuntu 16.04 xenial
numpy 1.13.1
scipy 0.19.1
matplotlib 2.0.2
pandas 0.20.3
Fri Aug 25 15:52:07 2017 AEST

In [2]:
import numpy as np
import cv2
import pandas as pd

import os
import sys

np.random.seed(0)

In [4]:
sys.path.append("/home/jack/Dev/BeeUnique/src")
from Processor import PathMetrics
from Processor.graphics import plot_path_bg, plot_heatmaps_bg_image, plot_histogram
from Processor.constants import *

In [5]:
experiment_directory = '/home/jack/Data/Caffeine_Unique_Tags'

pm = PathMetrics(experiment_directory)
pm.generate_night_day_bgs()

In [6]:
from scipy import ndimage
import math

In [15]:
def calc_heatmaps(list_bees, bg_image, file_name):
    all_coords_heatmap = np.zeros((NUM_Y_CELLS, NUM_X_CELLS))
    tag_class_presence_heatmap = np.zeros((NUM_Y_CELLS, NUM_X_CELLS))
    for bee in list_bees:
        for yx_coord in bee.slow_cells_visited.keys():
            if yx_coord[1] == NUM_X_CELLS:
                yx_coord = (yx_coord[0], NUM_X_CELLS - 1)
            all_coords_heatmap[yx_coord] += bee.slow_cells_visited[yx_coord]
            tag_class_presence_heatmap[yx_coord] += 1

    norm_all_coords_heatmap = all_coords_heatmap / all_coords_heatmap.sum()
    norm_tag_class_presence_heatmap = tag_class_presence_heatmap / tag_class_presence_heatmap.sum()

    all_coords_spread = calc_spread(norm_all_coords_heatmap)
    class_presence_spread = calc_spread(norm_tag_class_presence_heatmap)

    plot_heatmaps_bg_image(norm_all_coords_heatmap, bg_image, 100, 'All Coords Spread: ' + str(all_coords_spread), file_name +'_all_coords.png')
    plot_heatmaps_bg_image(norm_tag_class_presence_heatmap, bg_image, 100, 'Presence Spread: ' + str(class_presence_spread), file_name +'_presence.png')

    print(all_coords_spread, class_presence_spread)

def calc_spread(norm_heatmap):
    centre = ndimage.measurements.center_of_mass(norm_heatmap)
    spread = 0
    for y_c in range(0, norm_heatmap.shape[0]):
        for x_c in range(0, norm_heatmap.shape[1]):
            spread += calc_distance(x_c, y_c, centre[1], centre[0]) * norm_heatmap[y_c, x_c]
            
    return spread
            
def calc_distance(x1, y1, x2, y2):
    x_dist = (x2 - x1)
    y_dist = (y2 - y1)
    return math.sqrt(x_dist * x_dist + y_dist * y_dist)

In [14]:
for night_day in pm.night_day_tag_paths.keys():
    night_or_day_count = 0
    night_or_day_id = {'night': '_0_', 'day': '_1_'}
    for i in range(len(pm.night_day_tag_paths[night_day])):
        queen_bee = pm.night_day_tag_paths[night_day][i][QUEEN_CLASS]
        file_name = '/home/jack/' + str(night_or_day_count) + night_or_day_id[night_day] + night_day
        plot_path_bg(queen_bee, pm.night_day_bg_images['night'][i], file_name + '_bg.png')
        plot_histogram(queen_bee.distance_per_second, 'Distance per second', file_name + '_speeds.png')
        calc_heatmaps([queen_bee], pm.night_day_bg_images['night'][i], file_name)

        night_or_day_count += 1

(8.6074714656096596, 9.110522836829583)
(12.632877508797902, 13.970072522208016)
(17.493231652442844, 18.644469429923515)
(18.058856153502088, 19.806826306400549)


IndexError: index 80 is out of bounds for axis 1 with size 80